# Hierarchical Model for Payment Default Prediction

This notebook uses the **hierarchical architecture** that:
1. Groups invoices by user
2. Models the temporal trajectory of each user's payment behavior
3. Uses LSTM + Attention to capture sequential patterns

This is more appropriate for payment data than treating each invoice independently.

In [ ]:
import sys
import os
import torch
from datetime import datetime

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

from Model.core import run_pipeline
from Model.data_sequential import SequentialDataModule
from Model.network_hierarchical import HierarchicalModel
from Model.trainer_sequential import SequentialTrainer
from Configs.config import config
from Utils.logger import Logger

In [ ]:
# Setup paths
project_root = current_dir

run_id = datetime.now().strftime("%Y%m%d_%H%M%S") + "_hierarchical"
run_dir = os.path.join(project_root, "runs", run_id)
checkpoint_dir = os.path.join(run_dir, "checkpoints")
model_save_path = os.path.join(checkpoint_dir, "best_model.pth")

raw_path = os.path.join(project_root, config.paths.raw_data)
data_path = os.path.join(project_root, config.paths.train_data)

# Regenerate data with new features
if not os.path.exists(data_path):
    os.makedirs(os.path.dirname(data_path), exist_ok=True)
    run_pipeline(raw_path, data_path)
    print(f"Generated new training data: {data_path}")

logger = Logger(log_dir=run_dir)
print(f"Run directory: {run_dir}")

In [ ]:
# TensorBoard
runs_dir = os.path.join(project_root, "runs")
print(f"Starting TensorBoard on {runs_dir}...")
%load_ext tensorboard
%tensorboard --logdir "{runs_dir}" --port 6015 --reload_interval 30

In [ ]:
try:
    # Sequential Data Module - groups by user
    dm = SequentialDataModule(
        data_path,
        batch_size=32,  # Smaller batch for sequences
        num_workers=0,
        pin_memory=False,
        logger=logger,
        max_seq_len=50,  # Max invoices per user sequence
        min_seq_len=2    # Min invoices to include user
    )
    dm.prepare_data()
    
    # Hierarchical Model
    model = HierarchicalModel(
        embedding_dims=dm.emb_dims,
        n_cont=len(dm.cont_cols),
        hidden_dim=config.model.hidden_dim,
        n_invoice_layers=1,      # Layers for encoding single invoice
        n_sequence_layers=2,     # LSTM layers for temporal modeling
        n_heads=config.model.n_heads,
        dropout=config.model.dropout,
        use_temporal_attention=True  # Attention over history
    )
    
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(f"Hierarchical Model Parameters: {n_params:,}")
    
    # Sequential Trainer
    trainer = SequentialTrainer(
        model, dm,
        epochs=config.model.epochs,
        lr=config.model.lr,
        weight_decay=config.model.weight_decay,
        logger=logger,
        patience=config.model.patience,
        mixed_precision=config.model.mixed_precision,
        checkpoint_dir=checkpoint_dir,
        max_grad_norm=config.model.max_grad_norm,
        scheduler_factor=config.model.scheduler_factor,
        scheduler_patience=config.model.scheduler_patience,
        min_lr=config.model.min_lr,
        loss_type=config.model.loss_type,
        focal_alpha=config.model.focal_alpha,
        focal_gamma=config.model.focal_gamma
    )
    
    # Train
    best_model = trainer.fit()
    
    # Test
    test_metrics = trainer.test(dm.test_dataloader())
    
    # Save
    torch.save(best_model.state_dict(), model_save_path)
    print(f"\nTraining finished! Model saved to {model_save_path}")
    print(f"Test AUC-ROC: {test_metrics['auc_roc']:.4f}")
    print(f"Test AUC-PR: {test_metrics['auc_pr']:.4f}")

except Exception as e:
    logger.error(f"Failed: {e}")
    import traceback
    traceback.print_exc()
finally:
    logger.close()